In [92]:
import pandas as pd

In [186]:
df = pd.read_csv('/home/sandeep/Downloads/us_retail_sales.csv')

In [187]:
df['sales_month'] = pd.to_datetime(df['sales_month'])

In [188]:
df.head(5)

,sales_month,naics_code,kind_of_business,reason_for_null,sales
0,1992-01-01,441,Motor vehicle and parts dealers,NaN,29811.0
1,1992-01-01,4411,Automobile dealers,NaN,25800.0
2,1992-01-01,"4411, 4412",Automobile and other motor vehicle dealers,NaN,26788.0
3,1992-01-01,44111,New car dealers,NaN,24056.0
4,1992-01-01,44112,Used car dealers,NaN,1744.0


In [116]:
#SELF JOIN to get aggregate data and compute contribution to total

## Self-joins are tricky in pandas using the same name for both data frames will create head-aches
df2 = df[df.kind_of_business.str.contains('Automobile dealers|New car dealers', regex=True)].loc[:,['sales_month', 'kind_of_business',
                                                                                          'sales']]
df1 = df[df.kind_of_business.str.contains('Automobile dealers|New car dealers', regex=True)].merge(df2, on=['sales_month'], how='left')
d = df1.drop(['naics_code', 'reason_for_null', 'kind_of_business_y'], axis=1).groupby(by=['sales_month', 'kind_of_business_x', 'sales_x']).sum().sort_values(by=['sales_month'], ascending=True).reset_index()
d = d.rename(columns={'kind_of_business_x':'kind_of_business', 'sales_x':'sales', 'sales_y':'total_sales'})
d['pct_cont'] = d['sales']/d['total_sales']
d

,sales_month,kind_of_business,sales,total_sales,pct_cont
0,1992-01-01,Automobile dealers,25800.0,49856.0,0.517490
1,1992-01-01,New car dealers,24056.0,49856.0,0.482510
2,1992-02-01,Automobile dealers,27031.0,52072.0,0.519108
3,1992-02-01,New car dealers,25041.0,52072.0,0.480892
4,1992-03-01,Automobile dealers,30195.0,58213.0,0.518699
...,...,...,...,...,...
691,2020-10-01,New car dealers,87522.0,186000.0,0.470548
692,2020-11-01,Automobile dealers,89223.0,168492.0,0.529538
693,2020-11-01,New car dealers,79269.0,168492.0,0.470462
694,2020-12-01,Automobile dealers,101956.0,194344.0,0.524616


In [117]:
# --Aggregate data in year and compute each month's contribution - via window functions
df3 = df.assign(yr_sales=df.groupby(df.sales_month.dt.year)['sales'].transform('sum'))
df3 = df3.assign(cum_sum=df.sort_values(by=['sales_month', 'kind_of_business'])['sales'].cumsum())
df3 = df3.sort_values(by=['sales_month', 'kind_of_business'])
df3

,sales_month,naics_code,kind_of_business,reason_for_null,sales,yr_sales,cum_sum
44,1992-01-01,45299,All other gen. merchandise stores,NaN,2111.0,15710631.0,2111.0
11,1992-01-01,442299,All other home furnishings stores,Not Available,NaN,15710631.0,NaN
2,1992-01-01,"4411, 4412",Automobile and other motor vehicle dealers,NaN,26788.0,15710631.0,28899.0
1,1992-01-01,4411,Automobile dealers,NaN,25800.0,15710631.0,54699.0
5,1992-01-01,4413,"Automotive parts, acc., and tire stores",NaN,3023.0,15710631.0,57722.0
...,...,...,...,...,...,...,...
22576,2020-12-01,44511,Supermarkets and other grocery (except conveni...,NaN,64236.0,48758044.0,926145348.0
22559,2020-12-01,44112,Used car dealers,NaN,9568.0,48758044.0,926154916.0
22604,2020-12-01,45330,Used merchandise stores,NaN,1465.0,48758044.0,926156381.0
22598,2020-12-01,45291,Warehouse clubs and superstores,NaN,53210.0,48758044.0,926209591.0


In [184]:
# --Rolling average 12m - defining the window size aggregates over the fixed window
df4 = df
df4['r12m_sales'] = df4.groupby(by=['kind_of_business'])['sales'].transform(lambda x: x.rolling(12).mean())
df4[df4.kind_of_business == "Women's clothing stores"].head(20)

,sales_month,naics_code,kind_of_business,reason_for_null,sales,r12m_sales
29,1992-01-01,44812,Women's clothing stores,NaN,1873.0,NaN
94,1992-02-01,44812,Women's clothing stores,NaN,1991.0,NaN
159,1992-03-01,44812,Women's clothing stores,NaN,2403.0,NaN
224,1992-04-01,44812,Women's clothing stores,NaN,2665.0,NaN
289,1992-05-01,44812,Women's clothing stores,NaN,2752.0,NaN
354,1992-06-01,44812,Women's clothing stores,NaN,2424.0,NaN
419,1992-07-01,44812,Women's clothing stores,NaN,2373.0,NaN
484,1992-08-01,44812,Women's clothing stores,NaN,2657.0,NaN
549,1992-09-01,44812,Women's clothing stores,NaN,2560.0,NaN
614,1992-10-01,44812,Women's clothing stores,NaN,2755.0,NaN


In [197]:
# --MoM change calculation using LAG function
df5 = df
df5['prev_mnth_sales'] = df.sort_values(by=['kind_of_business','sales_month']).groupby(by=['kind_of_business'])['sales'].transform(lambda x: x.shift(1))
df5 = df5.sort_values(by=['kind_of_business','sales_month'])
df5['pct_mom_chng'] = (df5['sales']/df5['prev_mnth_sales']-1)*100
df5	

,sales_month,naics_code,kind_of_business,reason_for_null,sales,prev_mnth_sales,pct_mom_chng
44,1992-01-01,45299,All other gen. merchandise stores,NaN,2111.0,NaN,NaN
109,1992-02-01,45299,All other gen. merchandise stores,NaN,2149.0,2111.0,1.800095
174,1992-03-01,45299,All other gen. merchandise stores,NaN,2229.0,2149.0,3.722662
239,1992-04-01,45299,All other gen. merchandise stores,NaN,2427.0,2229.0,8.882907
304,1992-05-01,45299,All other gen. merchandise stores,NaN,2494.0,2427.0,2.760610
...,...,...,...,...,...,...,...
22324,2020-08-01,44812,Women's clothing stores,NaN,2386.0,2373.0,0.547830
22389,2020-09-01,44812,Women's clothing stores,NaN,2494.0,2386.0,4.526404
22454,2020-10-01,44812,Women's clothing stores,NaN,2634.0,2494.0,5.613472
22519,2020-11-01,44812,Women's clothing stores,NaN,2726.0,2634.0,3.492787


In [204]:
# --Period over period change using LAG
df6 = df
df6['prev_prd_12m'] = df6.sort_values(by=['kind_of_business', 'sales_month']).groupby(by=['kind_of_business', df6.sales_month.dt.month])['sales'].transform(lambda x: x.shift(1))
df6 = df6.sort_values(by=['kind_of_business','sales_month'])
df6.head(20)

,sales_month,naics_code,kind_of_business,reason_for_null,sales,prev_mnth_sales,prev_prd_12m
44,1992-01-01,45299,All other gen. merchandise stores,NaN,2111.0,NaN,NaN
109,1992-02-01,45299,All other gen. merchandise stores,NaN,2149.0,2111.0,NaN
174,1992-03-01,45299,All other gen. merchandise stores,NaN,2229.0,2149.0,NaN
239,1992-04-01,45299,All other gen. merchandise stores,NaN,2427.0,2229.0,NaN
304,1992-05-01,45299,All other gen. merchandise stores,NaN,2494.0,2427.0,NaN
369,1992-06-01,45299,All other gen. merchandise stores,NaN,2351.0,2494.0,NaN
434,1992-07-01,45299,All other gen. merchandise stores,NaN,2377.0,2351.0,NaN
499,1992-08-01,45299,All other gen. merchandise stores,NaN,2447.0,2377.0,NaN
564,1992-09-01,45299,All other gen. merchandise stores,NaN,2299.0,2447.0,NaN
629,1992-10-01,45299,All other gen. merchandise stores,NaN,2580.0,2299.0,NaN


In [185]:
df.head()

,sales_month,naics_code,kind_of_business,reason_for_null,sales,r12m_sales
0,1992-01-01,441,Motor vehicle and parts dealers,NaN,29811.0,NaN
1,1992-01-01,4411,Automobile dealers,NaN,25800.0,NaN
2,1992-01-01,"4411, 4412",Automobile and other motor vehicle dealers,NaN,26788.0,NaN
3,1992-01-01,44111,New car dealers,NaN,24056.0,NaN
4,1992-01-01,44112,Used car dealers,NaN,1744.0,NaN
...,...,...,...,...,...,...
22615,2020-12-01,NaN,GAFO(1),NaN,144871.0,103593.500000
22616,2020-12-01,NaN,Retail sales and food services excl motor vehi...,NaN,494937.0,414955.166667
22617,2020-12-01,NaN,"Retail and food services sales, total",NaN,611429.0,518699.916667
22618,2020-12-01,NaN,Retail sales and food services excl gasoline s...,NaN,574794.0,483078.500000
